# Ponizej krotki opis co przetestowalem do tej pory z algorytmow ML  
Na ta chwile opieram sie na Estra Trees -> dal mi najwyzsza dokladnosc jesli chodzi o accuracy.
W sekcji z treningiem modelu wzialem juz pod uwage, ze score jest liczony inaczej niz accuracy, 
a mianowicie log_loss, co zmienia troche postac rzeczy.

# Random Forest
wszystkie probki treningowe - bez tuningu  
testowe accuracy: 0.9817191330128261  
treningowe time: 44.32341456413269   ---   accuracy: 0.9991453923282256  

wszystkie - max_depth= 70, n_estimators= 100  
testowe: 0.9848913804199887  
treningowe: time: 469.39809942245483   ---   accuracy: 0.9999982415480004  

# Extra Trees
wszystkie probki treningowe - bez tuningu  
testowe accuracy: 0.9822326009966906  
treningowe time: 19.31137776374817   ---   accuracy: 1.0  

wszystkie - 'max_depth': None, 'n_estimators': 100   
testowe: 0.9837343190042941  
treningowe: time: 184.10754990577698   ---   1.0  

# Ada Boost
wszystkie probki treningowe - bez tuningu  
testowe accuracy: 0.4523828783045709  
treningowe time: 174.7924304008484   ---   0.4537878814522 

wszystkie - ??????  
testowe: ??????  
treningowe: time: ??????   ---   accuracy: ??????

# eXtreme Gradient Boosting
wszystkie probki treningowe - bez tuningu  
testowe accuracy: 0.6213947337879517   
treningowe time: 1652.873502254486   ---   0.622957997615539 

wszystkie - ??????  
testowe: ??????  
treningowe: time: ??????   ---   accuracy: ??????

# Gradient Boosting
wszystkie probki treningowe - bez tuningu  
testowe accuracy: 0.6797894077885356   
treningowe time: 3958.553186416626   ---   0.6825616073658037

wszystkie - ??????  
testowe: ??????  
treningowe: time: ??????   ---   accuracy: ??????

In [ ]:
import warnings
import time
warnings.filterwarnings('ignore')
import numpy as np
from matplotlib import pyplot as plt
import matplotlib
from matplotlib import colors as mcolors
from sklearn.ensemble.partial_dependence import partial_dependence, plot_partial_dependence
import pandas as pd
import sklearn as sk
from xgboost import XGBClassifier
from sklearn.preprocessing.imputation import Imputer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import IsolationForest
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
%matplotlib inline

# Wstepne przetwarzanie danych
Z racji, ze jest okolo 19GB danych testowych musialem podzielic je na mniejsze party 
(moje 32GB RAMu nie sa w stanie uciagnac wszystkiego jesli sie uwzgledni prace pythona,
ktory robi przewaznie kopie DataFramow do pamieci).

In [ ]:
dataset = pd.read_csv("C:\\Users\\Amadeusz\\Downloads\\all\\test_set.csv", 
                      dtype={"object_id":np.int64, "mjd":np.float64, "passband":np.int64,
                             "flux":np.float64, "flux_err":np.float64, "detected":np.int64})
print("Dataset loaded")

Podzial na 10 partow (text_n_part.csv)

In [ ]:
x = list(set(dataset["object_id"]))
n_parts = 10
dlugosc =  np.int(np.ceil(len(x)/n_parts))
for i, n in zip(range(0, len(x), dlugosc), range(1, 11)):
    start = time.time()
    dataset[dataset["object_id"].isin(x[i:i+dlugosc])].to_csv("..\\test_{}.csv".format(n))
    end = time.time()
    print("test_{}.csv saved - elapsed time: {}".format(n, end-start))
%xdel dataset
%xdel x

# Modele ML i statystyczne
Ogolnie czesc, ktora laduje modele z dysku o ile je mamy zapisane.
Jezeli nie, to na podstawie danych treningowych uzcy model i zapisuje na dysk.

Zawarty jest tez tutaj model wykrywania outlayers (wartosci odstajacych), w naszym przypadku novelty.
Pozniej bedzie potrzebny do klasyfikacji probek, ktore nie pasuja do schematu treningowego 
i nie powinny miec zadnej z wczesniejszych class/labels.

In [ ]:
import pickle
from sklearn.metrics import log_loss
try:
    # load the model from disk
    filename = '..\\ET_model_log_loss.sav'
    et = pickle.load(open(filename, 'rb'))
    print("Prediction model loaded.")
    
    # load the model from disk IsolationForest
    filename = '..\\ISF.sav'
    isf = pickle.load(open(filename, 'rb'))
    print("Outlayer model loaded")
    
#     # load the model from disk LocalOutlayerFactor
#     filename = '..\\LOF.sav'
#     lof = pickle.load(open(filename, 'rb'))
#     print("Outlayer model saved")
    
except FileNotFoundError as e:
    # load data ... (to be described)
    dataset = pd.read_csv("C:\\Users\\Amadeusz\\Downloads\\all\\training_set.csv")
    # load metadata, ... (to be described)
    meta_dataset = pd.read_csv('C:\\Users\\Amadeusz\\Downloads\\all\\training_set_metadata.csv')
    column_names = {6: "class_6", 15: "class_15", 16: "class_16", 42: "class_42", 52: "class_52", 53: "class_53",
                    62: "class_62", 64: "class_64", 65: "class_65", 67: "class_67", 88: "class_88", 90: "class_90",
                    92: "class_92", 95: "class_95"}
    # change labels according to sample submission example
    meta_dataset["target"] = list(map(lambda name: column_names[name], meta_dataset["target"]))
    # use imputer to search for NaN values and compute mean() values instead of NaN (column vise)
    mean_imputer = Imputer(missing_values='NaN', strategy='mean', axis=0)
    mean_imputer = mean_imputer.fit(meta_dataset.iloc[:,:-1]) # without last target column (imputer does not recognize string data)
    imputed_meta_dataset = mean_imputer.transform(meta_dataset.iloc[:,:-1].values)
    imputed_meta_dataset = pd.DataFrame(data=imputed_meta_dataset, columns=meta_dataset.iloc[:,:-1].columns.values)
    imputed_meta_dataset["target"] = meta_dataset.iloc[:,-1] # add last target column to imputed meta dataset
    # change object_id and ddf values back to int() ... imputer interprets all values as float()
    imputed_meta_dataset["object_id"] = list(map(lambda val: int(val), imputed_meta_dataset["object_id"]))
    imputed_meta_dataset["ddf"] = list(map(lambda val: int(val), imputed_meta_dataset["ddf"]))
    # merges two datasets, merge by group_id -> common key in both DataFrames
    training_dataset = pd.merge(dataset, imputed_meta_dataset)
    # check if training_dataset consists of any empty values
    columns_missing = [col for col in training_dataset.columns if training_dataset[col].isnull().any()]
    if columns_missing:
        print("Dataset has missing values in the following columns:\n{}".format(columns_missing))
    else:
        print("Dataset do not has any column with empty value.")
    # split data into training and test datasets (X and Y) ... data is randomly chosen
    test_size = 0.2
    seed = 7
    # description
    # train_test_split(X,Y,test_size,random_state)
    X_train, X_test, Y_train, Y_test = train_test_split(training_dataset.iloc[:,:-1], training_dataset.iloc[:,-1], 
                                                        test_size=test_size, random_state=seed)
    X_train = pd.DataFrame(data=X_train, columns=training_dataset.columns.values.tolist()[:-1])
    X_test = pd.DataFrame(data=X_test, columns=training_dataset.columns.values.tolist()[:-1])
    import time
    from sklearn.metrics import accuracy_score
    start = time.time()
    et = ExtraTreesClassifier(n_estimators=100)
    # .iloc[:,1:].values (removes object_id column from computing)
    et.fit(X_train.iloc[:,1:].values, Y_train)
    end = time.time()
    # accuracy score is not valid here ... accuracy means if we predict label correctly it is very good,
    # but not enough big predictions for other labels are ignored
    print("Extra Trees Elapsed Training time: {}   ---   Training accuracy score: {}".format(end - start, et.score(X_train.iloc[:,1:].values, Y_train)))
    start = time.time()
    predictions = et.predict(X_test.iloc[:,1:].values)
    end = time.time()
    print("Extra Trees Elapsed Test prediction time: {}   ---   Testing accuracy score: {}".format(end - start, accuracy_score(Y_test, predictions)))
    # compute predictions for each class for each row (sample)
    start = time.time()
    predicted_et = pd.DataFrame(et.predict_proba(X_test.iloc[:,1:].values), columns=et.classes_)
    end = time.time()
    print("Predict_proba computed in: {}".format(end-start))
    from sklearn.metrics import log_loss
    loss = log_loss(y_true=Y_test, y_pred=predicted_et.values, labels=predicted_et.columns.values.tolist())
    print("Log_loss: {}".format(loss))
    print("Precision: {}".format(np.exp(-loss)))

    # save model
    filename = '..\\ET_model_log_loss.sav'
    pickle.dump(et, open(filename, 'wb'))
    print("Prediction model saved.")
    
    # fit novelty detector model IsolationForest (~100 seconds computing time)
    start = time.time()
    isf = IsolationForest()
    isf.fit(np.ascontiguousarray(X_train.iloc[:,1:].values))
    end = time.time()
    print("after fit: {}".format(end-start))
    
    # save isf model
    filename = '..\\ISF.sav'
    pickle.dump(isf, open(filename, 'wb'))
    print("Outlayer estimator model saved.")
    
#     # scaling training data only for LocalOutlayerFactor(~1 second computing time)
#     start = time.time()
#     scaler = StandardScaler()
#     X_transformed = scaler.fit_transform(X_train.iloc[:,1:].values)
#     end = time.time()
#     print("after transformation: {}".format(end-start))
    
#     # fit novelty detector model LocalOutlayerFactor (~1800 seconds computing time)
#     start = time.time()
#     lof = LocalOutlierFactor(novelty=True)
#     lof.fit(np.ascontiguousarray(X_transformed))
#     end = time.time()
#     print("after fit: {}".format(end-start))
    
#     # save isf model
#     filename = '..\\LOF.sav'
#     pickle.dump(lof, open(filename, 'wb'))
#     print("Outlayer estimator model saved.")

https://www.kaggle.com/c/predict-closed-questions-on-stack-overflow/discussion/2499  
Wyjasnienie odnosnie log loss chociaz raczej nie do konca trafne jesli chodzi o ostatnie zdania (log5....itp).
Log_loss powinien schodzic jak najblizej wartosci zerowej wtedy prawdopodobienstwo, ze mamy dobre wyniki jest bliskie jedynki.

# Predict part
Klasyfikacja probek, plus utworzenie nowej klasy w zaleznosci czy probka jest odstajaca czy nie.

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
def predictions(predicted_dataframe, object_id):
    """Search rows with low prediction accuracy, if none of class was predicted higher then 0.6, classify as outlayer.
    Add class_99 column for outlayers"""
    columns = predicted_dataframe.columns.values.tolist()
    start = time.time()
    class_99 = np.any(predicted_dataframe[predicted_dataframe[columns] <= 0.6].apply(np.isnan), axis=1).apply(np.logical_not).apply(np.int)
    predicted_dataframe["class_99"] = class_99
    end = time.time()
    print("After search, elapsed time: {}".format(end-start))
    predicted_dataframe.insert(0,"object_id",object_id)
    
def predictions_ISF(predicted_dataframe, object_id, data, isf):
    """Use IsolationForest to classify if there is an outlayer, unseen label/class"""
    columns = predicted_dataframe.columns.values.tolist()
    n_samples = 1000000
    l_data = len(data)
    
    start = time.time()
    class_99 =  np.array([])
    for i in range(n_samples, l_data, n_samples): # IsolationForest cannot handle big datasets (32GB of RAM allocation is not enough)
        class_99 = np.append(class_99, isf.predict(data[i-n_samples:i])) # IsolationForest predicts if sample is outlayer (-1 or inlayer 1)
        print("Batch {} of {}, elapsed time: {}".format(i/n_samples, round(l_data/n_samples), time.time()-start))
    class_99 = np.append(class_99, isf.predict(data[l_data-l_data%n_samples:]))
    class_99[class_99 == -1] = 0 # change inlayers to 0 value
    predicted_dataframe["class_99"] = class_99 # append predictions with new class column
    end = time.time()
    
    print("After search, elapsed time: {}".format(end-start))
    predicted_dataframe.insert(0,"object_id",object_id)
    
%xdel dataset
%xdel meta_dataset
%xdel imputed_meta_dataset
%xdel training_dataset
%xdel X_train
%xdel X_test
%xdel Y_train
%xdel Y_test
print("After memory relocation")

meta_dataset = pd.read_csv('C:\\Users\\Amadeusz\\Downloads\\all\\test_set_metadata.csv')
mean_imputer = Imputer(missing_values='NaN', strategy='mean', axis=0)
mean_imputer = mean_imputer.fit(meta_dataset)
imputed_meta_dataset = mean_imputer.transform(meta_dataset.values)
imputed_meta_dataset = pd.DataFrame(data=imputed_meta_dataset, columns=meta_dataset.columns.values)
# change object_id and ddf values back to int() ... imputer interprets all values as float()
imputed_meta_dataset["object_id"] = list(map(lambda val: int(val), imputed_meta_dataset["object_id"]))
imputed_meta_dataset["ddf"] = list(map(lambda val: int(val), imputed_meta_dataset["ddf"]))

for i in range(1,11):
    dataset = pd.read_csv("..\\test_{}.csv".format(i), index_col=0)
    dataset_merged = pd.merge(dataset, imputed_meta_dataset, on="object_id")
    %xdel dataset
    object_id = dataset_merged["object_id"]
    start = time.time()
    predicted_et = pd.DataFrame(et.predict_proba(dataset_merged.iloc[:,1:].values), columns=et.classes_)
    end = time.time()
    print("Part {} Predict Elapsed time: {}".format(i, end-start))
    
    # IsolationForest method
    predictions_ISF(predicted_et, object_id, dataset_merged.iloc[:,1:].values, isf)
    # Manual method
#     predictions(predicted_et, object_id)
    
    start = time.time()
    predicted_et.to_csv("..\\predicted_et_n_{}.csv".format(i))
    end = time.time()
    print("Part {} Save to CSV Elapsed time: {}".format(i, end-start))
    %xdel dataset_merged
    %xdel predicted_et

# Grupowanie
Test dataset zawiera kilka probek na dany obiekt, 
z racji tego potrzeba wyestymowac z grupy probek dla obiektu do jakiej klasy nalezy.
Najlepiej dotad sprawdzila sie srednia z prawdopodobienstw dla danych klas.
Nie mam wyniku jeszcze dla sredniej z uwzglednieniem IsolationForest - postaram sie przeprowadzic obliczenia.

In [ ]:
# verified faster version (2.5 times faster then slower version version)
# def find_best_row(x):
#     """Finds row with maximum value in each group and returns that row."""
#     maximum = 0
#     return_row = 0
#     for ind, row in x.drop(["object_id"], axis=1).iterrows():
#         r = np.max(row.values)
#         if r > maximum:
#             maximum = r
#             return_row = row
#     return return_row

# def find_best_row(x):
#     """Finds row with maximum value in each group and returns that row."""
# #     s = time.time()
#     maximum = np.max(x.values)
# #     print("After max search, elapsed: {}".format(time.time()-s))
# #     s = time.time()
#     z = x[x[x.columns] == maximum].fillna(0)
# #     print("After fillna, elapsed: {}".format(time.time()-s))
#     return_row = 0
# #     s = time.time()
#     for ind, row in z.iterrows():
#         r = np.max(row.values)
#         if r == maximum:
#             return_row = row
#             break
# #     print("After for loop, elapsed: {}".format(time.time()-s))
#     return return_row  # leave only maximum prediction for row, other fill with zero

# slower version
# def find_best_row(x):
#     """Finds row with maximum value in each group and returns that row."""
#     index = x.drop(["object_id"], axis=1).T.max().idxmax()
#     return x.loc[[index]]

# compute grouped results for each object_id
for i in range(1,11):
    start = time.time()
    dataset = pd.read_csv("..\\predicted_et_n_{}.csv".format(i), index_col=0)
    print("predicted_et loaded, time elapsed: {}".format(time.time()-start))
#     grouped = dataset.groupby(by="object_id").apply(lambda x: find_best_row(x)).reset_index(level=1, drop=True).drop(["object_id"], axis=1)
#     print("Dataset grouped and best row found. time elapsed: {}".format(time.time() - start))
#     grouped = dataset.groupby(by="object_id").apply(lambda x: find_best_row(x.drop(["object_id"], axis=1)))    # returns only best row (for loop)
#     print("Dataset grouped and best row found. time elapsed: {}".format(time.time() - start))
    grouped = dataset.groupby(by="object_id").mean()    # returns mean of grouped rows
    grouped.to_csv("..\\grouped_{}.csv".format(i))
    %xdel grouped
    %xdel dataset
    end = time.time()
    print("Group {} saved. Elapsed time: {}".format(i, end-start))

# merge all parts into one dataset
for i in range(1,11):
    start = time.time()
    if i == 1:
        dataset = pd.read_csv("..\\grouped_{}.csv".format(i), index_col=0)
    else:
        dataset = pd.concat([dataset, pd.read_csv("..\\grouped_{}.csv".format(i))], ignore_index=True)
    end = time.time()
    print("Group {} added. Elapsed time: {}".format(i, end-start))

dataset.to_csv("..\\LSST_project_prediction_mean_ISF.csv", index=False)
%xdel dataset